In [1]:
import os
import pdfplumber
import re
import pandas as pd

invoice_folder = "Invoices"
output_file = "financial_summary.xlsx"

In [2]:
# --- Helper Functions ---

def ensure_pdf_extension(folder):
    for filename in os.listdir(folder):
        file_path = os.path.join(folder, filename)
        if os.path.isfile(file_path) and not filename.lower().endswith(".pdf"):
            try:
                with open(file_path, "rb") as f:
                    if f.read(5) == b"%PDF-":
                        os.rename(file_path, file_path + ".pdf")
                        print(f"Renamed {filename} → {filename}.pdf")
            except:
                continue

def extract_text(pattern, text):
    match = re.search(pattern, text)
    return match.group(1) if match else None

def extract_money(amount_str):
    """
    Convert a string like '-$81.96' or '$2,372.74' to float
    """
    if not amount_str:
        return 0.0
    return float(amount_str.replace("$","").replace(",",""))

def parse_line_to_category_amount(line):
    """
    Splits a statement line into clean category and amount.
    Assumes the LAST $xxx.xx is the amount.
    """
    money_match = re.search(r"(-?\$[\d,]+\.\d{2})", line)
    if not money_match:
        return None, None

    amount = extract_money(money_match.group(1))

    # Remove the amount from the line to get category
    category = line.replace(money_match.group(1), "").strip()

    # Clean extra symbols
    category = re.sub(r"\s{2,}", " ", category)
    category = category.rstrip("-").strip()

    return category, amount

def get_week_from_filename(filename):
    match = re.search(r"HF_W(\d+)", filename.upper())
    return match.group(1) if match else None

def extract_details_section(text):
    match = re.search(r"DETAILS(.*)", text, re.DOTALL)
    return match.group(1) if match else ""

# Parse PDF Detail Section

In [3]:
def parse_statement_long(text, week_from_filename=None):
    transactions = []

    year = extract_text(r"Accounting Year\s+(\d{4})", text)

    period_match = re.search(
        r"Payment Period\s+(\d{2}/\d{2}/\d{4})\s*-\s*(\d{2}/\d{2}/\d{4})",
        text
    )
    period_start = period_match.group(1) if period_match else None
    period_end = period_match.group(2) if period_match else None

    details_text = extract_details_section(text)
    lines = details_text.split("\n")

    for line in lines:
        line = line.strip()

        # Skip empty lines and Net Payment Total
        if not line or line.startswith("Net Payment Total"):
            continue

        # Capture dollar amount at end of line
        money_match = re.search(r"(-?\$[\d,]+\.\d{2})$", line)
        if not money_match:
            continue

        amount_str = money_match.group(1)
        amount = extract_money(amount_str)

        # Remove dollar amount from line for clean category
        line_clean = line.replace(amount_str, "").strip()

        # ---------- Gross Sales ----------
        if line_clean.startswith("Gross Sales"):
            units_match = re.search(r"Gross Sales\s+(\d+)", line_clean)
            units = units_match.group(1) if units_match else "N/A"
            category = f"Gross Sales ({units} units)"

        # ---------- Net Sales ----------
        elif line_clean.startswith("Net Sales"):
            category = "Net Sales"

        # ---------- Insurance ----------
        elif line_clean.startswith("Insurance"):
            category = "Insurance"

        # ---------- Credits ----------
        elif line_clean.startswith("Credits"):
            category = "Credits"

        # ---------- Deductions (dynamic & future-proof) ----------
        elif line_clean.startswith("Deductions"):
            category = line_clean.replace("Deductions", "").strip()

        # ---------- Fallback ----------
        else:
            category = line_clean

        transactions.append({
            "Week": week_from_filename,
            "Year": year,
            "Period Start": period_start,
            "Period End": period_end,
            "Category": category,
            "Amount": amount
        })

    return transactions

In [4]:
# -- Ensure PDFs
ensure_pdf_extension(invoice_folder)

In [5]:
# -- Process all PDFs
all_transactions = []

for pdf_file in os.listdir(invoice_folder):
    if not pdf_file.lower().endswith(".pdf"):
        continue

    week = get_week_from_filename(pdf_file)
    pdf_path = os.path.join(invoice_folder, pdf_file)

    with pdfplumber.open(pdf_path) as pdf:
        full_text = "\n".join(page.extract_text() or "" for page in pdf.pages)

    week_transactions = parse_statement_long(full_text, week)
    all_transactions.extend(week_transactions)

# -- Build Details DataFrame (long format) 
df_details = pd.DataFrame(all_transactions)

### Calculated Summary from `df_details`

In [6]:
# Build summary from details
summary_rows = []
for week, week_group in df_details.groupby("Week"):
    year = week_group["Year"].iloc[0]
    period_start = week_group["Period Start"].iloc[0]
    period_end = week_group["Period End"].iloc[0]

    net_sales = week_group.loc[week_group["Category"].str.contains("Net Sales"), "Amount"].sum()
    total_expenses = week_group.loc[week_group["Amount"] < 0, "Amount"].sum()
    net_profit = net_sales + total_expenses  # expenses are negative

    summary_rows.append({
        "Week": week,
        "Year": year,
        "Period Start": period_start,
        "Period End": period_end,
        "Net Sales (Calculated)": net_sales,
        "Total Expenses (Calculated)": total_expenses,
        "Net Profit (Calculated)": net_profit
    })

df_summary_calc = pd.DataFrame(summary_rows)

# Parse PDF Summary Section

In [7]:
def parse_statement_summary(text, week_from_filename=None):
    """
    Extracts Gross Sales, Net Sales, Insurance, Credits, Deductions, Net Payment from the summary section
    """
    summary_data = {"Week": week_from_filename}

    # Get SUMMARY section
    summary_text_match = re.search(r"SUMMARY(.*?)(DETAILS|$)", text, flags=re.DOTALL)
    summary_text = summary_text_match.group(1) if summary_text_match else text

    gross_sales = 0.0
    net_sales = 0.0
    credits = 0.0
    deductions_list = []
    net_payment = 0.0

    lines = summary_text.split("\n")
    for line in lines:
        line = line.strip()
        if not line or line.startswith("Entry Type"):
            continue

        match = re.search(r"([\w\s:%]+)\s+(-?\$[\d,]+\.\d{2})$", line)
        if match:
            category = match.group(1).strip()
            amount = float(match.group(2).replace("$", "").replace(",", ""))

            if category.startswith("Gross Sales"):
                gross_sales = amount
            elif category.startswith("Net Sales"):
                net_sales = amount
            elif category.startswith("Credits"):
                credits = amount
            elif category.startswith("Net Payment Total"):
                net_payment = amount
            else:
                deductions_list.append(amount)

    summary_data["Gross Sales (Statement)"] = gross_sales
    summary_data["Net Sales (Statement)"] = net_sales
    summary_data["Credits (Statement)"] = credits
    summary_data["Deductions (Statement)"] = sum(deductions_list)
    summary_data["Net Payment Total (Statement)"] = net_payment

    return summary_data

In [8]:
df_statement_summary_list = []

for pdf_file in os.listdir(invoice_folder):
    if not pdf_file.lower().endswith(".pdf"):
        continue

    week = get_week_from_filename(pdf_file)
    pdf_path = os.path.join(invoice_folder, pdf_file)

    with pdfplumber.open(pdf_path) as pdf:
        full_text = "\n".join(page.extract_text() or "" for page in pdf.pages)

    summary_data = parse_statement_summary(full_text, week)
    df_statement_summary_list.append(summary_data)

df_statement_summary = pd.DataFrame(df_statement_summary_list)

### Merge Calculated Summary and Statement Summary

In [9]:
df_compare = df_summary_calc.merge(
    df_statement_summary,
    on="Week",
    how="left"
)

df_compare["Net Profit Difference"] = (
    df_compare["Net Profit (Calculated)"] 
    - df_compare["Net Payment Total (Statement)"]
)

### Export to Excel with 2 tabs

In [10]:
with pd.ExcelWriter("financial_summary.xlsx", engine="openpyxl") as writer:
    df_details.to_excel(writer, sheet_name="Transaction Details", index=False)
    df_compare.to_excel(writer, sheet_name="Summary Comparison", index=False)

print("Excel exported: financial_summary.xlsx")

Excel exported: financial_summary.xlsx
